# Alpha Test Harness

Test ideas relating to alpha racing

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os

import numpy as np

from math import pi, cos, sqrt

from common_core import Printable, projdir, loadCsv

## Proximity Testing

Function to calculate the proximity between two points.

See the 'haversine_vs_pythagoras' notebook for a full comparison of the Haversine Formula vs Pythagorean Theorem.

The code here is very similar but makes use of pre-calculated cache values.

In [2]:
EARTH_RADIUS = 6371009


def estimateProximity(longitudes, y_offsets, x_scales, i1, i2):
    '''Estimate the Euclidean distance between two nearby points on a sphere using Pythagorean Theorem'''

    # Calculate distance north / south
    yDelta = y_offsets[i2] - y_offsets[i1]

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitudes[i2] - longitudes[i1])
    if longDelta > pi:
        longDelta -= 2 * pi

    # Calculate distance east / west
    xDelta = longDelta * x_scales[i2]

    # Apply Pythagorean theorem to determine the distance
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance

## Process Track

In [3]:
def processTrack(filename):
    '''Process the track'''

    # Load the track and calculate the extra fields
    track = loadCsv(filename)
    extras = calculateExtras(track)
    
    # Do a quick proximity test
    proximityTest(track, extras)

In [4]:
def calculateExtras(track):
    '''Calculate all of the extra fields for the track. These fields can be calculated on a per-trackpoint basis.'''

    extras = {}

    # Convert timestamps into datetimes
    extras['datetime'] = np.datetime64('1989-12-31T00:00') + track['timestamp'].astype('timedelta64[s]')
    
    # Convert latitude and longitude values from semicircles to radians
    extras['latitude'] = np.radians(track['position_lat'] * (180.0 / 2 ** 31))
    extras['longitude'] = np.radians(track['position_long'] * (180.0 / 2 ** 31))

    # Calculate distance north / south of the equator (metres)
    extras['y_offset'] = extras['latitude'] * EARTH_RADIUS

    # Calculate the scaling factor for distances east / west of the primary meridian
    extras['x_scale'] = EARTH_RADIUS * np.cos(extras['latitude'])
    
    # Calculate cumulative distance (m) using speed (m/s)
    extras['total_distance'] = track['speed'].cumsum()
    
    return extras

In [5]:
def proximityTest(track, extras):
    '''Development / testing only - Do a quick proximity test for a known alpha in the track from 2022-04-04'''

    # Avoid using the dictionary being accessed repeatedly
    datetimes = extras['datetime']
    latitudes = extras['latitude']
    longitudes = extras['longitude']
    y_offsets = extras['y_offset']
    x_scales = extras['x_scale']
    total_distances = extras['total_distance']

    # Find the start and the end of the fastest alpha
    i1 = np.where(datetimes == np.datetime64('2022-04-04T12:55:25'))[0][0]
    i2 = np.where(datetimes == np.datetime64('2022-04-04T12:56:08'))[0][0]
    
    # Calculate the proximity between the two points
    proximity = estimateProximity(longitudes, y_offsets, x_scales, i1, i2)
    
    print('Proximity = {:0.3f} metres'.format(proximity))

## Run Tests

In [6]:
if __name__ == '__main__':
    processTrack(os.path.join(projdir, 'sessions', '20220404', '20220404-apex-pro.csv'))

Proximity = 46.496 metres
